In [1]:
!pip install --upgrade gensim
!pip install -U gensim
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pickle as pk
import os
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.8/site-packages (4.0.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.8/site-packages (4.0.1)


In [2]:
data_dir = "./mimic-iii"
files_dir = "./dump_files"
model_dir = "./models"

In [3]:
df = pd.read_csv(os.path.join(files_dir, 'notes_disch.csv'))

In [4]:
df.columns

Index(['HADM_ID', 'TEXT'], dtype='object')

In [6]:
l = df.TEXT.tolist()

In [8]:
st =" ".join(l)
with open('disch_notes.txt', 'w') as f:
    f.write(st)

In [10]:
from gensim.models import Word2Vec
model = Word2Vec(df.TEXT, vector_size=100, window=5, min_count=3, workers=50, epochs=10)

In [11]:
model.save(os.path.join(model_dir, 'w2v-disch-100-5-3.bin'))

In [4]:
model = Word2Vec.load(os.path.join(model_dir, 'w2v-nurse_phys-100-5-3.bin'))

In [7]:
x = df.iloc[:,1]
y = df.iloc[:,2:]

In [9]:
y.shape

(8955, 6984)

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15,random_state=15)

In [13]:
WordVectorz=dict(zip(model.wv.index_to_key,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz).fit(x_train,y_train)
x_train = AverageEmbeddingVectorizer(WordVectorz).transform(x_train)

In [17]:
np.save(os.path.join(files_dir, 'x_train.npy'), x_train)
np.save(os.path.join(files_dir, 'x_test.npy'), x_test)
np.save(os.path.join(files_dir, 'y_train.npy'), y_train)
np.save(os.path.join(files_dir, 'y_test.npy'), y_test)
